In [1]:
import torchvision
import torch
from torchvision import transforms as T
from PIL import Image

In [2]:
print(torch.__version__)

1.10.1


In [3]:
model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=True)
model.eval()

# create the list of keypoints.
keypoints = ['nose','left_eye','right_eye',\
'left_ear','right_ear','left_shoulder',\
'right_shoulder','left_elbow','right_elbow',\
'left_wrist','right_wrist','left_hip',\
'right_hip','left_knee', 'right_knee', \
'left_ankle','right_ankle']

In [4]:
img_path =r"D:\Users\Camputer\source\repos\Skeleton\src\dataset\test\steth.jpg"
img = Image.open(img_path)

transform = T.Compose([T.ToTensor()])
img_tensor = transform(img)

output = model([img_tensor])[0]

D:\Users\Camputer\anaconda3\envs\skeleton-env\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import ImageDraw

def draw_keypoints_per_person(img, all_keypoints, all_scores, confs, keypoint_threshold=2, conf_threshold=0.9):
  # initialize a set of colors from the rainbow spectrum
  cmap = plt.get_cmap('rainbow')
  # create a copy of the image
  img_copy = img.copy()
  # pick a set of N color-ids from the spectrum
  color_id = np.arange(1,255, 255//len(all_keypoints)).tolist()[::-1]
  # iterate for every person detected
  for person_id in range(len(all_keypoints)):
    # check the confidence score of the detected person
    if confs[person_id]>conf_threshold:
      # grab the keypoint-locations for the detected person
      keypoints = all_keypoints[person_id, ...]
      # grab the keypoint-scores for the keypoints
      scores = all_scores[person_id, ...]
      # iterate for every keypoint-score
      for kp in range(len(scores)):
        # check the confidence score of detected keypoint
        if scores[kp]>keypoint_threshold:
          # convert the keypoint float-array to a python-list of integers
          keypoint = tuple(map(int, keypoints[kp, :2].detach().numpy().tolist()))
          # pick the color at the specific color-id
          color = tuple(np.asarray(cmap(color_id[person_id])[:-1])*255)
          # draw a circle over the keypoint location
          draw = ImageDraw.Draw(img_copy)
          x, y = tuple(p-s for p, s in zip(keypoint, (3, 3)))
          w, h = tuple(sum(i) for i in zip(keypoint, (3, 3)))
          shape = [(x, y), (w, h)]
          print(shape)
          draw.ellipse(shape, fill ="black", outline="black")
          draw.point(keypoint)
          #cv2.circle(img_copy, keypoint, 5, color, -1)

  return img_copy

In [7]:
keypoints_img = draw_keypoints_per_person(img, output["keypoints"], output["keypoints_scores"], output["scores"], keypoint_threshold=2)
keypoints_img.show()

[(489, 124), (495, 130)]
[(514, 98), (520, 104)]
[(466, 99), (472, 105)]
[(554, 103), (560, 109)]
[(431, 117), (437, 123)]
[(638, 285), (644, 291)]
[(344, 278), (350, 284)]
[(680, 503), (686, 509)]
[(279, 500), (285, 506)]
[(666, 659), (672, 665)]
[(308, 638), (314, 644)]
[(582, 712), (588, 718)]
[(392, 708), (398, 714)]
[(617, 1034), (623, 1040)]
[(324, 1013), (330, 1019)]
[(654, 1316), (660, 1322)]
[(273, 1298), (279, 1304)]


In [10]:
def get_limbs_from_keypoints(keypoints):
  limbs = [       
    [keypoints.index('right_eye'), keypoints.index('nose')],
    [keypoints.index('right_eye'), keypoints.index('right_ear')],
    [keypoints.index('left_eye'), keypoints.index('nose')],
    [keypoints.index('left_eye'), keypoints.index('left_ear')],
    [keypoints.index('right_shoulder'), keypoints.index('right_elbow')],
    [keypoints.index('right_elbow'), keypoints.index('right_wrist')],
    [keypoints.index('left_shoulder'), keypoints.index('left_elbow')],
    [keypoints.index('left_elbow'), keypoints.index('left_wrist')],
    [keypoints.index('right_hip'), keypoints.index('right_knee')],
    [keypoints.index('right_knee'), keypoints.index('right_ankle')],
    [keypoints.index('left_hip'), keypoints.index('left_knee')],
    [keypoints.index('left_knee'), keypoints.index('left_ankle')],
    [keypoints.index('right_shoulder'), keypoints.index('left_shoulder')],
    [keypoints.index('right_hip'), keypoints.index('left_hip')],
    [keypoints.index('right_shoulder'), keypoints.index('right_hip')],
    [keypoints.index('left_shoulder'), keypoints.index('left_hip')]
    ]
  return limbs

limbs = get_limbs_from_keypoints(keypoints)

In [9]:
def draw_skeleton_per_person(img, all_keypoints, all_scores, confs, keypoint_threshold=2, conf_threshold=0.9):
    # initialize a set of colors from the rainbow spectrum
    cmap = plt.get_cmap('rainbow')
    # create a copy of the image
    img_copy = img.copy()
    # check if the keypoints are detected
    if len(output["keypoints"])>0:
        # pick a set of N color-ids from the spectrum
        colors = np.arange(1,255, 255//len(all_keypoints)).tolist()[::-1]
        # iterate for every person detected
        for person_id in range(len(all_keypoints)):
            # check the confidence score of the detected person
            if confs[person_id]>conf_threshold:
            # grab the keypoint-locations for the detected person
                keypoints = all_keypoints[person_id, ...]

            # iterate for every limb 
            for limb_id in range(len(limbs)):
                # pick the start-point of the limb
                limb_loc1 = keypoints[limbs[limb_id][0], :2].detach().numpy().astype(np.int32)
                # pick the start-point of the limb
                limb_loc2 = keypoints[limbs[limb_id][1], :2].detach().numpy().astype(np.int32)
                # consider limb-confidence score as the minimum keypoint score among the two keypoint scores
                limb_score = min(all_scores[person_id, limbs[limb_id][0]], all_scores[person_id, limbs[limb_id][1]])
                # check if limb-score is greater than threshold
                if limb_score> keypoint_threshold:
                    # pick the color at a specific color-id
                    color = tuple(np.asarray(cmap(colors[person_id])[:-1])*255)
                    # draw the line for the limb
                    draw = ImageDraw.Draw(img_copy)
                    draw.line([tuple(limb_loc1), tuple(limb_loc2)], width=3)
                    #cv2.line(img_copy, tuple(limb_loc1), tuple(limb_loc2), color, 3)

    return img_copy


In [11]:
# overlay the skeleton in the detected person
skeletal_img = draw_skeleton_per_person(img, output["keypoints"], output["keypoints_scores"], output["scores"],keypoint_threshold=2)
skeletal_img.show()